In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from functools import partial
from extractors import *

In [2]:
trainFile = "train/trainingData.csv"
testFile = "test/testData.csv"
trainTransformed = "train/transformed.csv"
testTransformed = "test/transformed.csv"

In [3]:
buy  = 1
hold = 0
sell = -1

vmap = {"Buy":buy,"Hold":hold,"Sell":sell}
vmap_rev = {y:x for x,y in vmap.iteritems()}

t = 1
f = 0

buy_vmap =  {sell:f,hold:f,buy:t}
hold_vmap = {sell:f,hold:t,buy:f}
sell_vmap = {sell:t,hold:f,buy:f}


nmap = {
    buy:"Buy_n",
    hold:"Hold_n",
    sell:"Sell_n"
}

nmap_norm = {
    buy:"Buy_n_norm",
    hold:"Hold_n_norm",
    sell:"Sell_n_norm"
}

pmap = {
    buy:"Buy_p",
    hold:"Hold_p",
    sell:"Sell_p"
}

col_expert = "ExpertID"
col_symbol = "SymbolID"
col_prediction = "Prediction"
col_decision = "Decision"
col_trans = "TransID"

In [4]:
train = pd.DataFrame.from_csv(trainFile, sep=";",index_col=None)
test = pd.DataFrame.from_csv(testFile, sep=";",index_col=None)

In [5]:
train

,SymbolID,Recommendations,Decision
0,S94311056,"{3084,Buy,0.4603721,0}{3864,Buy,0.1864133,6}{2...",Sell
1,S175871,"{307,Buy,0.1847301,0}{307,Buy,0.1643529,40}",Hold
2,S1913464,"{3888,Buy,0.1612903,0}",Sell
3,S3682384,"{1702,Buy,0.1366866,0}",Buy
4,S94311056,"{3110,Buy,NA,0}{3117,Buy,0.2684415,8}{4527,Buy...",Buy
5,S4258978,"{3490,Sell,NA,0}",Buy
6,S3515147,"{3250,Hold,NA,0}{746,Hold,-0.07084785,0}",Sell
7,S3682384,"{1702,Hold,0.06903425,0}",Hold
8,S48162117,"{1346,Hold,0.1217359,0}{3519,Hold,-0.01690238,54}",Buy
9,S5131181,"{1735,Buy,1.288389,0}{3339,Buy,1.125963,8}{416...",Hold


In [6]:
test

,SymbolID,Recommendations
0,S1858788,"{848,Hold,0.05196047,0}{4357,Buy,0.2337333,34}..."
1,S30584221,"{1700,Hold,NA,0}{4349,Buy,0.185567,8}{1195,Hol..."
2,S104037,"{3832,Buy,0.3574661,0}"
3,S6419125,"{1645,Hold,NA,0}{530,Hold,-0.03942365,0}{1645,..."
4,S94311056,"{2208,Buy,0.5048619,0}{3084,Buy,0.2501929,0}{4..."
5,S3606780,"{8593,Buy,NA,0}{877,Hold,0.09051254,40}{3563,H..."
6,S3434273,"{4301,Buy,0.1432531,0}{1646,Hold,NA,35}"
7,S17646222,"{860,Hold,0.1673152,0}{3126,Buy,0.1172224,21}{..."
8,S448629,"{2096,Hold,NA,0}"
9,S11746101,"{860,Hold,0.1152416,0}{3302,Hold,0.1688312,26}..."


In [5]:
t_train = Extractor().transform(train)
t_train[col_decision] = train[col_decision].map(vmap)

In [19]:
t_train[col_decision] = train.iloc[t_train[col_trans]][col_decision].map(vmap).tolist()

In [20]:
t_train

,TransID,SymbolID,ExpertID,Prediction,Rate,TimeBefore,Decision
0,0.0,S94311056,3084,1,0.4603721,0,-1
1,0.0,S94311056,3864,1,0.1864133,6,-1
2,0.0,S94311056,25525,1,0.211129,7,-1
3,0.0,S94311056,342,1,0.1658611,9,-1
4,0.0,S94311056,1573,1,0.2515862,9,-1
5,0.0,S94311056,342,1,0.1766182,10,-1
6,0.0,S94311056,3104,1,0.1046623,11,-1
7,0.0,S94311056,941,1,0.3092294,11,-1
8,0.0,S94311056,2208,1,0.8181923,11,-1
9,0.0,S94311056,3105,1,NA,12,-1


In [6]:
t_train

,TransID,SymbolID,ExpertID,Prediction,Rate,TimeBefore,Decision
0,0.0,S94311056,3084,1,0.4603721,0,-1.0
1,0.0,S94311056,3864,1,0.1864133,6,0.0
2,0.0,S94311056,25525,1,0.211129,7,-1.0
3,0.0,S94311056,342,1,0.1658611,9,1.0
4,0.0,S94311056,1573,1,0.2515862,9,1.0
5,0.0,S94311056,342,1,0.1766182,10,1.0
6,0.0,S94311056,3104,1,0.1046623,11,-1.0
7,0.0,S94311056,941,1,0.3092294,11,0.0
8,0.0,S94311056,2208,1,0.8181923,11,1.0
9,0.0,S94311056,3105,1,NA,12,0.0


In [ ]:
t_train.to_csv(trainTransformed,index=False)

In [13]:
t_test = Extractor().transform(test)

In [14]:
t_test.to_csv(testTransformed,index=False)